In [1]:
#importing libraries
import numpy as np
import pandas as pd
import os
import itertools
from itertools import chain
os.getcwd()

'/home/linux/Desktop'

In [2]:
def PreparingDataset(filepath):
    data= pd.read_csv(filepath,header=None)
    data_dict={}
    for i in range(0,len(data)):
        l = data.iloc[i,0].split(',')
        data_dict.update({i:l})
    df = pd.DataFrame.from_dict(data_dict, orient='index')
    return df
    

In [3]:
def GeneratingC1AndL1(df, min_sup):
    unique_elements = []
    for i in df.columns:
        unique_elements.append(df[i].dropna().unique())
    flattened  = [val for sublist in unique_elements for val in sublist]
    item = list(set(flattened))
    c1= df.apply(pd.value_counts).sum(axis=1)
    l1 = c1[c1>=min_sup]
    return c1,l1
    

In [4]:
def GeneratindCandidatesAndFrequentSets(df,c1,l1,min_sup):
    candidates=[c1]
    frequent_set=[l1]
    l1_copy=l1
    size=2
    for j in range(len(df.columns)):
        c2={}
        comb=list(itertools.combinations(l1_copy.index, size))
        for i in comb: 
            if j==0:
                comb_prev=list(chain(*itertools.combinations(i,size-1)))
            else:
                comb_prev=list(itertools.combinations(i,size-1))

            if set(comb_prev)<=set(l1.index.tolist()):
                freq=df.isin(i).sum(axis=1)[df.isin(i).sum(axis=1)>1].count()
                c2.update({i:freq}) #generation of candidates 
            else:
                continue
        if(len(c2)==0):
            break        
        c2 = pd.Series(c2)        
        l2=c2[c2>=min_sup]  #generation of frequent Sets
        candidates.append(c2)
        frequent_set.append(l2)
        l1=l2
        size+=1
    return candidates, frequent_set    
    

In [11]:
def RulesGeneration(c1,frequent_set,confidence):
    freq_set = frequent_set[-1]
    freq_set_dict = freq_set.to_dict()
    for i in freq_set_dict.keys():
        freq_set_dict[i]=freq_set_dict[i]/c1[i[0]]
    rules = pd.Series(freq_set_dict)
    rules=rules[rules>=confidence]
    return rules

In [12]:
def main():
    filepath = input('Enter the filname with filepath')
    threshold = float(input('Enter the minimum threshold'))
    confidence = float(input('Enter the minimum confidence'))
    df = PreparingDataset(filepath)
    min_sup = threshold*len(df)
    c1,l1 = GeneratingC1AndL1(df,min_sup)
    candidates,frequent_set = GeneratindCandidatesAndFrequentSets(df,c1,l1,min_sup)
    rule = RulesGeneration(c1,frequent_set,confidence)
    rule = rule.to_dict()
    for i in rule.keys():
        print(i)
    

In [13]:
main()

Enter the filname with filepath/home/linux/Desktop/dataset.csv
Enter the minimum threshold0.2
Enter the minimum confidence0.5
('BISCUIT', 'BREAD')
('COFFEE', 'CORNFLAKES')
('COFFEE', 'SUGER')
('MAGGI', 'TEA')
